In [12]:
# Importing necessary Libraries

import requests
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [13]:
# The list of Top 20 CryptoCurenncy

coin_list = ['BTC', 'ETH', 'XRP', 'ADA', 'USDT', 'DOGE', 'XLM', 'DOT', 'UNI', 'LINK', 'USDC', 'BCH', 'LTC', 'GRT', 'ETC', 
             'FIL', 'AAVE', 'ALGO', 'EOS']

In [14]:
# Creating DataFrame

main_df = pd.DataFrame()

In [15]:
for coin in coin_list:
    coin_df = pd.DataFrame()
    df = pd.DataFrame(index=[0])
    
    # Defining the Start Date and End Date
    datetime_end = datetime(2021, 7, 2, 0, 0)
    datetime_check = datetime(2021, 7, 1, 0, 0)
    
    while len(df) > 0:
        if datetime_end == datetime_check:
            break
        
        datetime_start = datetime_end - relativedelta(hours = 12)
        
        #Api for the scrapping
        url = 'https://production.api.coindesk.com/v2/price/values/'+ coin +'?start_date='+datetime_start.strftime("%Y-%m-%dT%H:%M") + '&end_date=' + datetime_end.strftime("%Y-%m-%dT%H:%M") + '&ohlc=true'
        
        #we are using the request to fetch the data from the api in the json format and then storing it into the dataframe.
        temp_data = requests.get(url).json()
        df = pd.DataFrame(temp_data['data']['entries'])
        df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close']
        
        # To handle the Missing Data
        insert_ids_list = [np.nan]
        
        '''In numerous occasions, I observed that CoinDesk did not collect every minute of data. 
        According to observation, in a normal circumstance, the timestamp would have a discrepancy
        value of 60000 for 1 minute. As a result, I can quickly tell that the missing period is the
        time gap if the row difference for the timestamp is larger than 60000. To deal with it, a
        hot-deck imputation procedure is used. To put it another way, the nearest minute data will 
        be substituted for the missing one.
        '''
        
        while len(insert_ids_list) > 0:
            timestamp_checking = np.array(df['Timestamp'][1:]) - np.array(df['Timestamp'][:-1])
            insert_ids_list = np.where(timestamp_checking!= 60000)[0]
            if len(insert_ids_list) > 0:
                print(str(len(insert_ids_list)) + ' mismatched.')
                insert_ids = insert_ids_list[0]
                temp_df = df.iloc[insert_ids.repeat(int(timestamp_checking[insert_ids]/60000)-1)].reset_index(drop=True)
                temp_df['Timestamp'] = [temp_df['Timestamp'][0] + i*60000 for i in range(1, len(temp_df)+1)]
                df = df.loc[:insert_ids].append(temp_df).append(df.loc[insert_ids+1:]).reset_index(drop=True)
                insert_ids_list = insert_ids_list[1:]
                
        
        #adding datetime and symbol to dataframe
        df = df.drop(['Timestamp'], axis=1)
        df['Datetime'] = [datetime_end - relativedelta(minutes=len(df)-i) for i in range(0, len(df))]
        coin_df = df.append(coin_df)
        datetime_end = datetime_start
        
    coin_df['Symbol'] = coin
    main_df = main_df.append(coin_df)


1 mismatched.
1 mismatched.
1 mismatched.
1 mismatched.
1 mismatched.
1 mismatched.
1 mismatched.
1 mismatched.
1 mismatched.
1 mismatched.
1 mismatched.
1 mismatched.
1 mismatched.
1 mismatched.
1 mismatched.
1 mismatched.
1 mismatched.
1 mismatched.
1 mismatched.


In [16]:
main_df = main_df[['Datetime', 'Symbol', 'Open', 'High', 'Low', 'Close']].reset_index(drop=True)
main_df

,Datetime,Symbol,Open,High,Low,Close
0,2021-07-01 00:00:00,BTC,35049.045484,35056.817222,34991.326658,34993.994267
1,2021-07-01 00:01:00,BTC,34995.349202,35030.583041,34989.167695,34991.266464
2,2021-07-01 00:02:00,BTC,34991.950939,34994.431719,34929.671801,34952.450112
3,2021-07-01 00:03:00,BTC,34954.642324,34980.657519,34954.642324,34961.649826
4,2021-07-01 00:04:00,BTC,34960.350436,34996.245435,34960.350436,34976.586395
...,...,...,...,...,...,...
27355,2021-07-01 23:55:00,EOS,3.945295,3.945295,3.936671,3.939916
27356,2021-07-01 23:56:00,EOS,3.939917,3.940801,3.933735,3.933735
27357,2021-07-01 23:57:00,EOS,3.933920,3.934184,3.928797,3.933092
27358,2021-07-01 23:58:00,EOS,3.933122,3.939846,3.933122,3.939811


In [17]:
main_df.to_csv('main_df.csv', index=False)